# 11 MR analysis core

**Origin:** `1_1_running_MR.ipynb`  
**This annotated version was generated on:** 2025-10-13 06:41

**What this notebook does (high level):**  
- Run Mendelian Randomization (MR) using eQTL/pQTL instruments within DHS. Includes harmonization and sensitivity analyses.

**How to use:**  
1. Review the markdown notes before each code cell.  
2. Adjust input/output paths as needed for your environment.  
3. Run cell-by-cell to reproduce artifacts for downstream steps.

---


**Step 1:** Load tabular data (summary stats / annotations).

In [ ]:
# run in command line R 

# bulk tissue 

library(data.table)
library(dplyr)
library(TwoSampleMR)
library(future.apply)

setwd('/mnt/f/10_osteo_MR/MR_ready/')

run_mr_bulk <- function(exposure, outcome, gene_col="gene", out_prefix) {
  # 1. Subset outcome to only relevant SNPs
  all_exp_snps <- unique(exposure$SNP)
  outcome <- outcome[SNP %in% all_exp_snps]
  genes <- unique(exposure[[gene_col]])
  
  # 2. Per-gene MR function (suitable for parallelization)
  mr_one_gene <- function(g) {
    exp_sub <- exposure[exposure[[gene_col]] == g]
    out_sub <- outcome[SNP %in% exp_sub$SNP]
    if (nrow(exp_sub) < 2 | nrow(out_sub) < 2) return(NULL)
    exp_dat <- suppressWarnings(format_data(
      as.data.frame(exp_sub), type="exposure", snp_col="SNP", beta_col="beta", se_col="se",
      effect_allele_col="effect_allele", other_allele_col="other_allele",
      phenotype_col=gene_col, pval_col="pval"
    ))
    out_dat <- suppressWarnings(format_data(
      as.data.frame(out_sub), type="outcome", snp_col="SNP", beta_col="beta", se_col="se",
      effect_allele_col="effect_allele", other_allele_col="other_allele",
      pval_col="pval"
    ))
    dat <- suppressWarnings(harmonise_data(exp_dat, out_dat, action=1))
    if (nrow(dat) < 2) return(NULL)
    mr_res <- tryCatch({
      suppressWarnings(mr(dat, method_list=c("mr_ivw", "mr_egger_regression", "mr_weighted_median")))
    }, error = function(e) NULL)
    if (!is.null(mr_res) && nrow(mr_res) > 0) {
      mr_res$gene <- g
      mr_res$n_instruments <- nrow(dat)
      return(mr_res)
    }
    return(NULL)
  }
  
  # 3. Parallel over genes
  plan(multisession, workers = max(1, parallel::detectCores()-1))
  res_list <- future_lapply(genes, mr_one_gene)
  all_results <- dplyr::bind_rows(res_list)
  
  # 4. Save results
  data.table::fwrite(all_results, paste0(out_prefix, ".tsv"), sep="\t")
  message("Wrote: ", out_prefix, ".tsv")
}

# ---- Example Usage for Each Exposure ----

out_osteo <- fread("outcome_osteo_within_wb_DHS.tsv")

# eQTLGen
exp_eqtlgen <- fread("exposure_eqtlgen_dhs_index.tsv")
run_mr_bulk(exp_eqtlgen, out_osteo, gene_col="gene", out_prefix="MR_result_eqtlgen_osteo")

                       
# GTEx whole blood eQTL
exp_gtex <- fread("exposure_gtex_whole_blood_eqtl_dhs_index.tsv")
run_mr_bulk(exp_gtex, out_osteo, gene_col="gene", out_prefix="MR_result_gtex_osteo")


# UKB-PPP pQTL
exp_ukbppp_pqtl <- fread("exposure_ukbppp_pqtl_dhs_index.tsv")
run_mr_bulk(exp_ukbppp_pqtl, out_osteo, gene_col="gene", out_prefix="MR_result_pqtl_ukbppp_osteo")

                
                       
# deCode pQTL
exp_decode_pqtl <- fread("exposure_pqtl_decode_dhs_index.tsv")
run_mr_bulk(exp_decode_pqtl, out_osteo, gene_col="gene", out_prefix="MR_result_pqtl_decode_osteo")


